In [1]:
from pyspark import SparkContext,HiveContext
from pyspark.sql.functions import *
from pyspark.sql.window import Window

print("Lets do the 3rd question ")

Lets do the 3rd question 


In [3]:
'''
Q3: Find top 5 teams based on the number of matches they won
'''
teamDf = sqlContext.sql( ''' select * from fujitsu.team''').cache()
matchDf = sqlContext.sql( ''' select winner_team_api_id,win_counter,row_number() over (order by win_counter desc) as winning_rank
from (select winner_team_api_id, sum(winning_counter) as win_counter from (select case when (home_team_goal > away_team_goal) then home_team_api_id
when (home_team_goal = away_team_goal) then 0 else away_team_api_id
end as winner_team_api_id , 1 as winning_counter from  fujitsu.match_new) innerQuery1 where winner_team_api_id !=0 group by winner_team_api_id  ) innerQuery2 ''' ).cache()

In [6]:
matchDfTop5Team = matchDf.where( col('winning_rank') <6).join(teamDf,col('winner_team_api_id') == col('team_api_id'))
matchDfTop5Team.sort('winning_rank').drop('id').drop('winner_team_api_id').show()

+-----------+------------+-----------+----------------+-----------------+---------------+
|win_counter|winning_rank|team_api_id|team_fifa_api_id|   team_long_name|team_short_name|
+-----------+------------+-----------+----------------+-----------------+---------------+
|        234|           1|       8634|             241|     FC Barcelona|            BAR|
|        227|           2|       8633|             243|   Real Madrid CF|            REA|
|        218|           3|       9925|              78|           Celtic|            CEL|
|        193|           4|       9823|              21| FC Bayern Munich|            BMU|
|        192|           5|      10260|              11|Manchester United|            MUN|
+-----------+------------+-----------+----------------+-----------------+---------------+



In [13]:
'''
Q4: Analyze if particular country has “Home Advantage” (wins more often) over others
'''
matchnewDf = sqlContext.sql( '''select win_home_team_api_id,win_away_team_api_id, winning_home_cnt,winning_away_cnt
from (select win_home_team_api_id,count(win_home_team_api_id) as winning_home_cnt from (
select case when (home_team_goal > away_team_goal) then home_team_api_id else 0 end as 
win_home_team_api_id from fujitsu.match_new ) innerQuery where win_home_team_api_id !=0 group by win_home_team_api_id) inerQuer2
inner join 
(select win_away_team_api_id,count(win_away_team_api_id) as winning_away_cnt from (
select case when (home_team_goal < away_team_goal) then away_team_api_id else 0 end as 
win_away_team_api_id from fujitsu.match_new ) innerQuery where win_away_team_api_id !=0 group by win_away_team_api_id ) 
innerQuery3 on win_home_team_api_id = win_away_team_api_id ''').cache()

In [15]:
homeWinCnt = matchnewDf.where (col('winning_home_cnt') > col('winning_away_cnt')).sort('winning_home_cnt')

In [16]:
homeWinCnt.show()

+--------------------+--------------------+----------------+----------------+
|win_home_team_api_id|win_away_team_api_id|winning_home_cnt|winning_away_cnt|
+--------------------+--------------------+----------------+----------------+
|              108893|              108893|               2|               1|
|                6631|                6631|               3|               1|
|                6351|                6351|               3|               2|
|                7992|                7992|               3|               2|
|                4064|                4064|               3|               2|
|                6547|                6547|               3|               2|
|                9746|                9746|               4|               3|
|                9878|                9878|               4|               2|
|                6269|                6269|               4|               2|
|                9798|                9798|               4|    